In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Define the list of currencies you need
CURRENCIES = ['EUR', 'GBP', 'HKD', 'HUF', 'ILS', 'INR', 'JPY', 'KES', 'KRW', 'KWD', 'MXN', 'NOK', 'NZD', 'PLN', 'QAR', 'RON', 'RUB', 'SEK', 'SGD', 'THB', 'TRY', 'TWD', 'USD']

# Define the start and end dates
end_date = datetime.now()
start_date = end_date - timedelta(days=365 * 10)

def fetch_historical_rates(date, currencies):
    """Fetch historical exchange rates from Frankfurter.app for a specific date."""
    url = f"https://api.frankfurter.app/{date}?from=USD&to={','.join(currencies)}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes
        return response.json()['rates']
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from Frankfurter.app for date {date}: {e}")
        return {}

def get_historical_exchange_rates():
    """Fetch historical exchange rates for the last 10 years."""
    historical_rates = []
    current_date = start_date
    
    while current_date <= end_date:
        date_str = current_date.strftime('%Y-%m-%d')
        rates = fetch_historical_rates(date_str, CURRENCIES)
        if rates:
            rates['Date'] = date_str
            historical_rates.append(rates)
        current_date += timedelta(days=1)
    
    # Create a DataFrame
    df = pd.DataFrame(historical_rates)
    df.set_index('Date', inplace=True)
    return df

if __name__ == "__main__":
    df = get_historical_exchange_rates()
    if not df.empty:
        print(df.head())
        # Save the DataFrame to a CSV file
        df.to_csv('historical_exchange_rates.csv')
    else:
        print("Failed to fetch historical exchange rates.")